In [ ]:
import os
from skimage import io
import matplotlib.pyplot as plt
#import skimage.morphology
import skimage.segmentation
from skimage.measure import regionprops
#from scipy.ndimage.morphology import distance_transform_edt
import numpy as np
from scipy import ndimage


In [ ]:
# Path
ground_truth_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\Ground_Truth'

# Import test image
image_name = 'C - 02.tif'
img_filename = os.path.join(ground_truth_path, image_name)
image = io.imread(img_filename)

# Get labels and plot
image = skimage.morphology.label(image)
plt.imshow(image)

In [ ]:
# Method to erode label image
def erode_labels(image, erosion_iterations):
    # create empty list where the eroded masks can be saved to
    list_of_eroded_masks = []
    regions = regionprops(image)
    def erode_mask(image_labels, label_id, erosion_iterations):
        
        only_current_label_id = np.where(image_labels == label_id, 1, 0)
        eroded = ndimage.binary_erosion(only_current_label_id, iterations = erosion_iterations)
        relabeled_eroded = np.where(eroded == 1, label_id, 0)
        return(relabeled_eroded)

    for i in range(len(regions)):
        label_id = regions[i].label
        list_of_eroded_masks.append(erode_mask(image, label_id, erosion_iterations))

    # convert list of numpy arrays to stacked numpy array
    final_array = np.stack(list_of_eroded_masks)

    # max_IP to reduce the stack of arrays, each containing one labelled region, to a single 2D np array. 
    final_array_labelled = np.sum(final_array, axis = 0)
    return(final_array_labelled)

In [ ]:
#eroded = erode_labels(image, 10)
#plt.imshow(eroded)

In [ ]:
# Method to get semantic labels of objects and edges
def semantic_edges (image, distance):
    expanded_labels = skimage.segmentation.expand_labels(label_image=image, distance=distance)
    eroded_labels = erode_labels(image, distance)

    edges = expanded_labels - eroded_labels
    objects = expanded_labels - edges

    edges_semantic = np.where(edges > 0, 1, 0)
    object_semantic = np.where(objects > 0, 2, 0)
    semantic_labels = edges_semantic + object_semantic
    return semantic_labels

In [ ]:
# Get semantic image
image_semantic = semantic_edges(image, 2)

# Show a crop
image_semantic_crop = image_semantic[800:1200, 800:1200]
plt.imshow(image_semantic_crop)
plt.colorbar()
plt.show()